# Advanced `pandas`

The following notebook is dedicated to more advanved opeartions in Pandas:

- `split-apply-combine` pipeline,
- operations on string columns (string operations, replacement),
- joins on Pandas dataframes.

In [ ]:
%pylab inline
plt.style.use("bmh")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
titanic_train = pd.read_csv("train.csv", index_col="PassengerId")
titanic_test = pd.read_csv("test.csv", index_col="PassengerId")
titanic = pd.concat([titanic_train, titanic_test], sort=False)

In [ ]:
titanic.head()

# Joining Pandas dataframes (`JOIN` in Pandas)

We start with a synthetic example:

In [ ]:
a = pd.DataFrame(np.arange(8).reshape((4,2)),
                 columns=["a", "b"],
                 index=["a", "b", "a", "b"])
b = pd.DataFrame(10 + np.arange(4).reshape((4,-1)),
                 columns=["d"],
                 index=["d", "b", "c", "b"])

In [ ]:
a

In [ ]:
b

In [ ]:
a.join(b) # default is left join

In [ ]:
a.join(b, how="inner")

In [ ]:
a

In [ ]:
b

In [ ]:
b.join(a, how="right")

In [ ]:
a.join(b, how="outer")

We can also perform join operation on multi-indexed dataframes:

In [ ]:
c = pd.DataFrame(np.arange(8).reshape((4,2)),
                 columns=["a", "b"],
                 index=pd.MultiIndex.from_tuples([("a", "A"), ("b", "E"), ("a", "Y"), ("b", "R")],
                                                 names=("lower", "upper")))

In [ ]:
c

In [ ]:
a

In [ ]:
c.join(a, on="lower")  # This one will fail

In [ ]:
c.join(a, on="lower", rsuffix="_right", lsuffix="_left")

# Joining dataframes for EDA

## Problem: get (almost) all couples on board 

In [ ]:
titanic[["Name", "Sex"]].head()

We start by noting the pattern: married females are listed as `<FAMILY_NAME>, Mrs. <HUSBANDS_FIRST_NAME> (<WIFES_FULL_NAME>)`. Let's play with it a bit:

In [ ]:
family_names = (titanic
                .replace(re.compile(r'\s+\(.*\)'), '')
                .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex"]]

In [ ]:
family_names

Removing wife's names that appear in brackets:

In [ ]:
titanic.replace(re.compile(r'\s+\(.*\)'), '')

In [ ]:
family_names

We can now get passenger IDs and husbands names of all married women (note that not all of these husbands are on board!)

In [ ]:
family_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]

In [ ]:
family_names.head()

In [ ]:
family_names.shape[0]

We now want to join this back to original dataframe (a very common pattern if you need some **pairs**):

In [ ]:
family_names.reset_index().set_index("Name")["PassengerId"]

In [ ]:
couples = (titanic.join(family_names
                        .reset_index()
                        .set_index("Name")["PassengerId"],
                        on="Name", how="inner"))
couples

Note, that there is no colission on `PassengerId` **column** because husband PassengerId is an index!

In [ ]:
couples.rename({"PassengerId":"PassengerId_Spouse"},
               axis=1, inplace=True)

In [ ]:
couples.head()

In [ ]:
couples = couples.join(titanic[["Name", "Age"]],
                       on="PassengerId_Spouse", rsuffix="_Spouse")

In [ ]:
couples

In [ ]:
titanic.Pclass.value_counts()

In [ ]:
couples.Pclass.value_counts()

In [ ]:
couples.Sex.value_counts()

In [ ]:
((couples.Age - couples.Age_Spouse)
 .groupby(couples.Pclass)
 .agg(["min", "max", "mean", "median", "std", "count", "size"]))

In [ ]:
couples[(couples.Age - couples.Age_Spouse)<0][["PassengerId_Spouse", "Name", "Age", "Name_Spouse", "Age_Spouse"]]

In [ ]:
titanic.loc[742]

In [ ]:
titanic.loc[988]

Although it's only heuristics, and we may need to dig deeper (e.g., to find some uncommon naming patterns), this is already something. Think on which features you may add to quantify a passenger (say, `is wife/husband on board?`, which may complement `SibSp`).

Think on how you may find entire **families**, and which features you may extract by knowing those. EDA is about your data driven creativity, so - play with it.

P. S. **not a single loop** above.

### Intermezzo: on self-joins

In [ ]:
cabin_counts = titanic.Cabin.value_counts()
cabin_counts[cabin_counts>1]

In [ ]:
cabin_counts = cabin_counts[cabin_counts>1]

In [ ]:
cabins = (titanic
          .loc[titanic.Cabin.isin(cabin_counts.index),
               ["Name", "Cabin"]]
          .reset_index())

In [ ]:
cabins.merge(cabins, on="Cabin", suffixes=("_first", "_second"))

In [ ]:
companions = cabins.merge(cabins, on="Cabin", suffixes=("_first", "_second"))
companions = companions[companions.PassengerId_first != companions.PassengerId_second]

In [ ]:
companions

We can now clean this up and get another interesting source of information (`travelling with a family member in the same cabin?`, etc.).